In [1]:
%matplotlib inline
import sys
import random
import operator
import pandas as pd
import numpy as np
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
from sklearn import preprocessing, metrics, ensemble, neighbors, linear_model, tree, model_selection
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import manifold, decomposition
from sklearn.random_projection import GaussianRandomProjection, SparseRandomProjection

/home/dj/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

In [3]:
train_y=train['target']

In [4]:
all_data = pd.concat((train.loc[:,'num_var_1':'cat_var_42'],
                      test.loc[:,'num_var_1':'cat_var_42']))

In [5]:
col_to_drop=['cat_var_24', 'cat_var_25', 'cat_var_26', 'cat_var_27', 'cat_var_28', 'cat_var_29', 'cat_var_30', 
             'cat_var_31', 'cat_var_32', 'cat_var_33', 'cat_var_34', 'cat_var_35', 'cat_var_36', 'cat_var_37', 
             'cat_var_38', 'cat_var_39', 'cat_var_40', 'cat_var_41','cat_var_42', 'num_var_3']

In [6]:
all_data=all_data.drop(col_to_drop, 1)

In [7]:
all_data['cat_var_1']=all_data['cat_var_1'].fillna('Nan')
all_data['cat_var_3']=all_data['cat_var_3'].fillna('Nan')
all_data['cat_var_6']=all_data['cat_var_6'].fillna('Nan')
all_data['cat_var_8']=all_data['cat_var_8'].fillna('Nan')

In [8]:
encoder=all_data.groupby('cat_var_1').size()
encoder=encoder/len(all_data)
all_data['cat_var_1_freq']=all_data.cat_var_1.map(encoder)

encoder=all_data.groupby('cat_var_2').size()
encoder=encoder/len(all_data)
all_data['cat_var_2_freq']=all_data.cat_var_2.map(encoder)

encoder=all_data.groupby('cat_var_3').size()
encoder=encoder/len(all_data)
all_data['cat_var_3_freq']=all_data.cat_var_3.map(encoder)

encoder=all_data.groupby('cat_var_6').size()
encoder=encoder/len(all_data)
all_data['cat_var_6_freq']=all_data.cat_var_6.map(encoder)

encoder=all_data.groupby('cat_var_7').size()
encoder=encoder/len(all_data)
all_data['cat_var_7_freq']=all_data.cat_var_7.map(encoder)

encoder=all_data.groupby('cat_var_8').size()
encoder=encoder/len(all_data)
all_data['cat_var_8_freq']=all_data.cat_var_8.map(encoder)

encoder=all_data.groupby('cat_var_10').size()
encoder=encoder/len(all_data)
all_data['cat_var_10_freq']=all_data.cat_var_10.map(encoder)

encoder=all_data.groupby('cat_var_13').size()
encoder=encoder/len(all_data)
all_data['cat_var_13_freq']=all_data.cat_var_13.map(encoder)

encoder=all_data.groupby('cat_var_14').size()
encoder=encoder/len(all_data)
all_data['cat_var_14_freq']=all_data.cat_var_14.map(encoder)

In [9]:
all_data.head()

,num_var_1,num_var_2,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_1,cat_var_2,cat_var_3,cat_var_4,...,cat_var_23,cat_var_1_freq,cat_var_2_freq,cat_var_3_freq,cat_var_6_freq,cat_var_7_freq,cat_var_8_freq,cat_var_10_freq,cat_var_13_freq,cat_var_14_freq
0,2.302632e-08,0.040182,1.800000e-07,2.302632e-08,2.368421e-08,1.115205e-08,Nan,ce,db,ep,...,0,0.039514,0.667156,0.023058,0.016274,0.998761,0.135110,0.030682,0.025584,0.711649
1,7.965789e-06,0.157872,2.105000e-06,2.769737e-07,7.965789e-06,2.433058e-06,da,tn,zl,tn,...,0,0.290023,0.295931,0.009840,0.289718,0.998761,0.135110,0.036715,0.299744,0.711649
2,7.828947e-08,0.089140,3.550000e-07,4.671053e-08,1.052632e-07,4.276014e-07,gf,ce,gs,tn,...,0,0.379122,0.667156,0.004132,0.378800,0.998761,0.379122,0.092515,0.380945,0.711649
3,7.894737e-08,0.227239,1.050000e-06,1.381579e-07,2.190789e-07,1.848054e-08,Nan,ce,fy,ep,...,0,0.039514,0.667156,0.002698,0.005679,0.998761,0.019988,0.029980,0.016051,0.034106
4,3.321053e-06,0.160410,2.105000e-06,2.769737e-07,3.340789e-06,2.152983e-06,da,tn,zn,tn,...,0,0.290023,0.295931,0.000298,0.289718,0.998761,0.135110,0.049561,0.299744,0.711649


In [10]:
def encoder(data):
    '''Map the categorical variables to numbers to work with scikit learn'''
    for col in data.columns:
        if data.dtypes[col] == "O":
            le = preprocessing.LabelEncoder()
            data[col]=le.fit_transform(data[col])
    return data

In [11]:
all_data=encoder(all_data)

In [12]:
train_X = all_data[:len(train)]
test_X = all_data[len(train):]

In [13]:
test_id=test['transaction_id']

In [64]:
def create_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    for i, feat in enumerate(features):
        outfile.write('{0}\t{1}\tq\n'.format(i,feat))
    outfile.close()

def runXGB(train_X, train_y, test_X, test_y=None, test_X2=None, feature_names=None, seed_val=0, rounds=500, dep=10, eta=0.1):
    params = {}
    params["objective"] = "binary:logistic"
    params['eval_metric'] = 'auc'
    params["eta"] = eta
    params["subsample"] = 0.7
    params["min_child_weight"] = 1
    params["colsample_bytree"] = 0.7
    params["max_depth"] = dep

    params["silent"] = 1
    params["seed"] = seed_val
    #params["max_delta_step"] = 2
    #params["gamma"] = 0.5
    num_rounds = rounds

    plst = list(params.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist, early_stopping_rounds=100, verbose_eval=20)
    else:
        xgtest = xgb.DMatrix(test_X)
        model = xgb.train(plst, xgtrain, num_rounds)

    if feature_names is not None:
        create_feature_map(feature_names)
        model.dump_model('xgbmodel.txt', 'xgb.fmap', with_stats=True)
        importance = model.get_fscore(fmap='xgb.fmap')
        importance = sorted(importance.items(), key=operator.itemgetter(1), reverse=True)
        imp_df = pd.DataFrame(importance, columns=['feature','fscore'])
        imp_df['fscore'] = imp_df['fscore'] / imp_df['fscore'].sum()
        imp_df.to_csv("imp_feat.txt", index=False)

    pred_test_y = model.predict(xgtest, ntree_limit=model.best_ntree_limit)
    pred_test_y2 = model.predict(xgb.DMatrix(test_X2), ntree_limit=model.best_ntree_limit)

    loss = 0
    if test_y is not None:
        loss = metrics.roc_auc_score(test_y, pred_test_y)
        return pred_test_y, loss, pred_test_y2
    else:
        return pred_test_y, loss, pred_test_y2

In [37]:
def runLGB(train_X, train_y, test_X, test_y=None, test_X2=None, feature_names=None, seed_val=0, rounds=500, dep=10, eta=0.1):
    params = {}
    params["objective"] = "binary"
    params['metric'] = 'auc'
    params["max_depth"] = dep
    params["min_data_in_leaf"] = 64
    params["learning_rate"] = eta
    params["bagging_fraction"] = 0.7
    params["feature_fraction"] = 0.7
    params["bagging_freq"] = 5
    params["bagging_seed"] = seed_val
    params["verbosity"] = 0
    #params["reg_alpha"] = 20
    #params["reg_lambda"] = 20
    num_rounds = rounds

    plst = list(params.items())
    lgtrain = lgb.Dataset(train_X, label=train_y)

    if test_y is not None:
        lgtest = lgb.Dataset(test_X, label=test_y)
        model = lgb.train(params, lgtrain, num_rounds, valid_sets=[lgtest], early_stopping_rounds=100, verbose_eval=20)
    else:
        lgtest = lgb.DMatrix(test_X)
        model = lgb.train(params, lgtrain, num_rounds)

    pred_test_y = model.predict(test_X, num_iteration=model.best_iteration)
    pred_test_y2 = model.predict(test_X2, num_iteration=model.best_iteration)

    loss = 0
    if test_y is not None:
        loss = metrics.roc_auc_score(test_y, pred_test_y)
        print(loss)
        return pred_test_y, loss, pred_test_y2
    else:
        return pred_test_y, loss, pred_test_y2

In [48]:
from catboost import CatBoostClassifier
def runCatB(train_X, train_y, test_X, test_y=None, test_X2=None, depth=8):
    model = CatBoostClassifier(
                                iterations = 1000,
                                learning_rate = 0.1,
                                depth = depth,
                                od_type='Iter',
                                od_wait=100,
                                rsm=0.7,
                                l2_leaf_reg=3,
                                eval_metric = 'AUC', 
                                verbose=True,
                                random_seed=42)
    
    model.fit(train_X, train_y, eval_set=(test_X, test_y))#, plot=True)
    train_preds = model.predict_proba(train_X)[:,1]
    test_preds = model.predict_proba(test_X)[:,1]
    test_preds2 = model.predict_proba(test_X2)[:,1]
    test_loss = 0
    if test_y is not None:
        train_loss = metrics.roc_auc_score(train_y, train_preds)
        test_loss = metrics.roc_auc_score(test_y, test_preds)
        print("Depth : ", depth)
        print("Train and Test loss : ", train_loss, test_loss)
    return test_preds, test_loss, test_preds2

In [17]:
def runET(train_X, train_y, test_X, test_y=None, test_X2=None, depth=20, leaf=10, feat=0.8):
    model = ensemble.ExtraTreesClassifier(
                                        n_estimators = 100,
                                        criterion = 'gini',
                                        max_depth = depth,
                                        min_samples_split = 10,
                                        min_samples_leaf = leaf,
                                        warm_start = True,
                                        max_features =  feat,
                                        #min_impurity_split = 0.1,
                                        n_jobs = -1,
                                        verbose = 3,
                                        random_state = 0)
    model.fit(train_X, train_y)
    train_preds = model.predict_proba(train_X)[:,1]
    test_preds = model.predict_proba(test_X)[:,1]
    test_preds2 = model.predict_proba(test_X2)[:,1]
    test_loss = 0
    if test_y is not None:
        train_loss = metrics.roc_auc_score(train_y, train_preds)
        test_loss = metrics.roc_auc_score(test_y, test_preds)
        print("Depth, leaf, feat : ", depth, leaf, feat)
        print("Train and Test loss : ", train_loss, test_loss)
    return test_preds, test_loss, test_preds2

In [42]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
def runADA(train_X, train_y, test_X, test_y=None, test_X2=None, depth=6):
    model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=depth, max_features=0.7, min_samples_leaf=5),
                               n_estimators = 300,
                               learning_rate = 0.2,
                               random_state=42)
    
    model.fit(train_X, train_y)#, eval_set=(test_X, test_y))#, plot=True)
    train_preds = model.predict_proba(train_X)[:,1]
    test_preds = model.predict_proba(test_X)[:,1]
    test_preds2 = model.predict_proba(test_X2)[:,1]
    test_loss = 0
    if test_y is not None:
        train_loss = metrics.roc_auc_score(train_y, train_preds)
        test_loss = metrics.roc_auc_score(test_y, test_preds)
        print("Depth : ", depth)
        print("Train and Test loss : ", train_loss, test_loss)
    return test_preds, test_loss, test_preds2

In [69]:
kf = model_selection.StratifiedKFold(n_splits=5, shuffle=True, random_state=2018)
cv_scores = []
pred_test_full = 0
pred_val_full = np.zeros(train_X.shape[0])
for dev_index, val_index in kf.split(train_X, train_y):
    dev_X, val_X = train_X.iloc[dev_index,:], train_X.iloc[val_index,:]
    dev_y, val_y = train_y[dev_index], train_y[val_index]

        
    pred_val, loss, pred_test = runXGB(dev_X, dev_y, val_X, val_y, test_X, rounds=5000, feature_names=dev_X.columns.tolist())
            
    pred_val_full[val_index] = pred_val
    pred_test_full = pred_test_full + pred_test
    cv_scores.append(loss)
    print(cv_scores)
pred_test_full /= 5.
print(metrics.roc_auc_score(train_y, pred_val_full))

[0]	train-auc:0.701043	test-auc:0.699736
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
[20]	train-auc:0.758102	test-auc:0.73217
[40]	train-auc:0.791475	test-auc:0.732451
[60]	train-auc:0.820601	test-auc:0.730105
[80]	train-auc:0.840729	test-auc:0.730549
[100]	train-auc:0.858166	test-auc:0.730872
[120]	train-auc:0.871303	test-auc:0.73048
Stopping. Best iteration:
[25]	train-auc:0.767073	test-auc:0.734038

[0.73403839041269503]
[0]	train-auc:0.701608	test-auc:0.697055
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 100 rounds.
[20]	train-auc:0.76059	test-auc:0.727775
[40]	train-auc:0.793423	test-auc:0.730925
[60]	train-auc:0.816584	test-auc:0.730136
[80]	train-auc:0.84192	test-auc:0.730972
[100]	train-auc:0.863632	test-auc:0.730667
[120]	train-auc:0.876143	test-auc:0.731899
[140]	train-auc:0.889013	test-auc:0.732

In [70]:
out_df = pd.DataFrame({"transaction_id":test_id})
out_df["target"] = pred_test_full
out_df.to_csv("XGB.csv", index=False)

In [71]:
train['XGB_val_full']=pred_val_full

In [56]:
train_X['XGB_val_full']=train['XGB_val_full']
train_X['LGM_val_full']=train['LGM_val_full']
train_X['ADA_val_full']=train['ADA_val_full']
train_X['CatB_val_full']=train['CatB_val_full']
train_X['ET_val_full']=train['ET_val_full']

/home/dj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/dj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/dj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

In [58]:
a=pd.read_csv('pred_test_XGB_Final.csv')
b=pd.read_csv('pred_test_LGM_Final.csv')
c=pd.read_csv('pred_test_ADA_Final.csv')
d=pd.read_csv('pred_test_CatB_Final.csv')
e=pd.read_csv('pred_test_ET_Final.csv')

test_X['XGB_val_full']=a['target']
test_X['LGM_val_full']=b['target']
test_X['ADA_val_full']=c['target']
test_X['CatB_val_full']=d['target']
test_X['ET_val_full']=e['target']

/home/dj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/home/dj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/dj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [61]:
train_X.drop(['ADA_val_full'], 1, inplace=True)
test_X.drop(['ADA_val_full'], 1, inplace=True)

/home/dj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/home/dj/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [ ]:
runLGB(dev_X, dev_y, val_X, val_y, test_X, rounds=5000)
runCatB(dev_X, dev_y, val_X, val_y, test_X)
runET(dev_X, dev_y, val_X, val_y, test_X)
runADA(dev_X, dev_y, val_X, val_y, test_X)

In [65]:
train_X.head()

,num_var_1,num_var_2,num_var_4,num_var_5,num_var_6,num_var_7,cat_var_1,cat_var_2,cat_var_3,cat_var_4,...,cat_var_6_freq,cat_var_7_freq,cat_var_8_freq,cat_var_10_freq,cat_var_13_freq,cat_var_14_freq,XGB_val_full,LGM_val_full,CatB_val_full,ET_val_full
0,2.302632e-08,0.040182,1.800000e-07,2.302632e-08,2.368421e-08,1.115205e-08,0,3,75,0,...,0.016274,0.998761,0.135110,0.030682,0.025584,0.711649,0.061212,0.055136,0.059754,0.035387
1,7.965789e-06,0.157872,2.105000e-06,2.769737e-07,7.965789e-06,2.433058e-06,66,50,607,1,...,0.289718,0.998761,0.135110,0.036715,0.299744,0.711649,0.053016,0.075225,0.051236,0.053073
2,7.828947e-08,0.089140,3.550000e-07,4.671053e-08,1.052632e-07,4.276014e-07,128,3,158,1,...,0.378800,0.998761,0.379122,0.092515,0.380945,0.711649,0.066105,0.068473,0.064990,0.044393
3,7.894737e-08,0.227239,1.050000e-06,1.381579e-07,2.190789e-07,1.848054e-08,0,3,140,0,...,0.005679,0.998761,0.019988,0.029980,0.016051,0.034106,0.051330,0.058330,0.055494,0.054163
4,3.321053e-06,0.160410,2.105000e-06,2.769737e-07,3.340789e-06,2.152983e-06,66,50,609,1,...,0.289718,0.998761,0.135110,0.049561,0.299744,0.711649,0.061850,0.071204,0.056037,0.072610


In [67]:
train_X=train_X.loc[:,'num_var_1':'cat_var_14_freq']

In [68]:
test_X=test_X.loc[:,'num_var_1':'cat_var_14_freq']